# <center>__MÉTODOS NUMÉRICOS__</center>
## <center>__PROJETO DA UNIDADE 2 - PageRank com Autodecomposições__</center>

#### <center>__DUPLA: JOÃO VICTOR NEGREIROS DA SILVA e LUCAS BIVAR FONSÊCA TAVARES__</center>

<div class="alert alert-block alert-info">
1. INTRODUÇÃO
</div>

O PageRank é um algoritmo desenvolvido em 1998 por Sergey Brin e Larry Page (por isso o nome "Page" Rank) enquanto cursavam a Universidade de Stanford. Esse algoritmo foi o primeiro utilizado pela ferramenta de busca da Google para classificação de páginas, e permaneceu assim por longos anos. Apesar de hoje a empresa também utilizar outros algoritmos como esse, o PageRank ainda é o mais conhecido. De forma resumida, esse algoritmo atua como uma ferramenta que ajuda a avaliar a importância de um site em relação a outros.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/PageRank-hi-res.png/300px-PageRank-hi-res.png" />

Para construir uma métrica de PageRank, primeiros temos que visualizar as páginas da internet em um formato de rede (ou grafo), onde cada nó representa uma página e cada ligação (ou aresta) corresponde uma citação de uma página para outra. O algoritmo então atribui um valor para cada nó com base nas ligações que ela recebe e de quem recebe. Dessa forma, quanto mais citações uma página recebe, maior seu valor na métrica do PageRank, e caso essas citações venham de páginas de valor alto, sua métrica tende a crescer mais ainda.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/PageRanks-Example.svg/400px-PageRanks-Example.svg.png" />

<div class="alert alert-block alert-info">
2. DESCRIÇÃO DO PROBLEMA
</div>

Neste notebook, o problema apresentado é a classificação (ou rankeameto) de páginas da Wikipédia, de forma que possamos identificar de forma rápida quais as mais relevantes. Para isso será utilzado o algoritmo "PageRank" (utilizado pela Google) aliado a outros métodos (que serão discutidos na seção posterior) que identificam a "importância" da página a partir de suas ligações com outras páginas.

<div class="alert alert-block alert-info">
3. MÉTODOS APLICADOS À SOLUÇÃO
</div>

Para solucionar o problema, iremos utilizar alguns métodos de decomposição com matrizes.

### Decomposição de valores singulares (SVD)

Maneira de fatoração de uma matriz real ou complexa na seguinte forma:

$M = U \Sigma V^*$

Onde:
- $U$ = Matriz unitária $m ₓ m$
- $\Sigma$ = Matriz retangular diagonal m×n com números reais não-negativos na diagonal
- $V^*$ = Conjugada transposta de V

### Decomposição de Eigen
Para entender esse método, é importante apresentar os conceitos de autovetores e autovalores.

Dada uma matriz quadrada $A$ de ordem $n ₓ n$, dizemos que um número real $\lambda $ é um **autovalor** de $A$ quando existe um vetor não nulo $\overrightarrow{v}$ tal que:

$A\overrightarrow{v} = \lambda\overrightarrow{v}$

Neste caso, $\overrightarrow{v}$ é um dito um **autovetor** de $A$ associado a $\lambda$.

A decomposição de Eigen fatora uma matriz representando-a em termos de seus autovalores e autovetores:

$A = Q \Lambda Q^{-1}$

Onde:
- $A$ = matriz quadrada $n ₓ n$
- $Q$ = matriz quadrada $n ₓ n$ onde a n-ésima coluna corresponde ao n-ésimo autovetor de A.
- $\Lambda$ = matriz diagonal onde os elementos da diagonal correspondem aos autovalores, $\Lambda_{ii} = \lambda_{i}$

### Power Method (Método das Potências)
O método das potências é um algoritmo para calcular autovalores. Dada uma matriz A, o algoritmo produzirá um autovalor ($\lambda$) e um autovetor ($\overrightarrow{v}$), tal que:

$A\overrightarrow{v} = \lambda\overrightarrow{v}$

Também é conhecido como iteração de Von Mises

<div class="alert alert-block alert-info">
4. IMPLEMENTAÇÃO
</div>

Para a implementação dos métodos necessários, será utilizado algumas libs auxiliares para lidar com debug, manipulação de arquivos e requisições http:
- os
- pickle
- tqdm
- bz2
- urllib

\
Além dessas também será utilizada bibliotecas bem conhecidas de manipulação de grandes quantidades de dados:
- numpy
- scipy

\
O conjunto de dados utilizado serão os arquivos de redirect e links do DBpedia apenas do idioma Inglês (en). No entanto foi adicionado um limite de 1.000.000 nas iterações na base de dados, para não prolongar o tempo de execução das funções.

## Imports necessários

In [ ]:
import os
import pickle
import numpy as np
import tqdm.notebook as tqdm
from bz2 import BZ2File
from scipy import sparse
from urllib.request import urlopen

## Constantes úteis

In [ ]:
# Dataset com links da Wikipedia
URL_BASE = 'http://downloads.dbpedia.org/3.8/en/'

# Dados de URLS que redirecionam para outras URLS
REDIRECTS_FILENAME = 'redirects_en.nt.bz2'

# Paginas que possuem links para outras paginas
PAGE_LINKS_FILENAME = 'page_links_en.nt.bz2'


filenames = [REDIRECTS_FILENAME, PAGE_LINKS_FILENAME]

## Baixando os dados

In [ ]:
for filename in filenames:
    if not os.path.exists(filename):
        print("Downloading '%s', please wait..." % filename)
        open(filename, 'wb').write(urlopen(URL_BASE+filename).read())

In [ ]:
DBPEDIA_RESOURCE_PREFIX_LEN = len("http://dbpedia.org/resource/")
SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)

# Removendo primeira e ultima linha por motivos de limpeza de dados
def get_lines(filename): return (line.split() for line in BZ2File(filename))

## Criando dicionário
Essa estrutura vai guardar as informações dos redirecionamentos da origem para o destino final

In [ ]:
def get_redirect(targ, redirects):
    seen = set()
    while True:
        transitive_targ = targ
        targ = redirects.get(targ)
        if targ is None or targ in seen: break
        seen.add(targ)
    return transitive_targ


def get_redirects(redirects_filename):
    redirects = {}
    res = {}
    lines = get_lines(redirects_filename)
    cont = 0
    for arr in tqdm.tqdm(lines):
        key = arr[0][SLICE].decode("utf-8")
        res[key] = get_redirect(arr[2][SLICE].decode("utf-8"), redirects)

        # Limitando iterações devido a grande quantidade de dados
        if cont == 1_000_000:
          break
        cont += 1
    return res


redirects = get_redirects(REDIRECTS_FILENAME)

0it [00:00, ?it/s]

In [ ]:
def add_item(lst, redirects, index_map, item):
    k = item[SLICE]
    redirect_target = redirects.get(k)
    res = True
    if redirect_target is None:
        res = False
        redirect_target = k
    update_index_map = index_map.setdefault(redirect_target, len(index_map))
    lst.append(update_index_map)
    return res

In [ ]:
# Computing the integer index map
index_map = dict()  # links->IDs
lines = get_lines(PAGE_LINKS_FILENAME)
source, destination, data = [], [], []
cont = 0
for l, split in tqdm.tqdm(enumerate(lines)):
    add_item(source, redirects, index_map, split[0].decode("utf-8"))
    add_item(destination, redirects, index_map, split[2].decode("utf-8"))
    data.append(1)

    # Limitando iterações devido a grande quantidade de dados
    if cont == 1_000_000:
      break
    cont += 1

len(index_map)

0it [00:00, ?it/s]

370075

O dicionário final conta com mais de 370 mil índices. Vamos criar uma cópia do mesmo e verificar algumas informações

In [ ]:
index_map_copy = index_map.copy()

Verificando o id da página "FIFA_World_Cup"

In [ ]:
FIFA_WORLD_CUP_PAGE_ID = index_map_copy['FIFA_World_Cup']
FIFA_WORLD_CUP_PAGE_ID

124417

Buscando as 5 primeiras páginas que possuem links para a página "FIFA_World_Cup"

In [ ]:
[i for i, x in enumerate(source) if x == FIFA_WORLD_CUP_PAGE_ID][:5]

[901222, 901223, 901224, 901225, 901226]

Vamos utilizar a primeira página, cujo id vale 901222, como exemplo

In [ ]:
for page_name, index in index_map_copy.items():
    if index == destination[901222]:
        print(page_name)

FIFA_World_Cup_Trophy


Ao acessar o <a href="https://en.wikipedia.org/wiki/FIFA_World_Cup_Trophy">link</a> da página "FIFA_World_Cup_Trophy" vemos que realmente ocorre uma citação para a página "FIFA_World_Cup"

<div class="alert alert-block alert-info">
5. CASOS DE USO
</div>

## 5.1 Utilizando base de dados EN (carregados previamente)

Com base nos arquivos carregados nas etapas anteriores, criaremos uma matriz esparsa dos dados:

In [ ]:
n=len(data)
sparse_matrix = sparse.coo_matrix((data, (destination,source)), shape=(n,n), dtype=np.float32)
sparse_matrix_csr = sparse_matrix.tocsr()

Para poupar tempo e consumo de memória, o autor salva a matriz esparsa e o dicionário de itens em dois arquivos, evitando assim recompilá-los sempre.

In [ ]:
SPARSE_MATRIX_EN_FILENAME = 'sparse_matrix_EN.pkl'
INDEX_MAP_EN_FILENAME = 'index_map_EN.pkl'

pickle.dump(sparse_matrix_csr, open(SPARSE_MATRIX_EN_FILENAME, 'wb'))
pickle.dump(index_map_copy, open(INDEX_MAP_EN_FILENAME, 'wb'))

Para carregar os arquivos basta descomentar e executar a célula abaixo:

In [ ]:
# sparse_matrix_csr = pickle.load(open(SPARSE_MATRIX_EN_FILENAME, 'rb'))
# index_map_copy = pickle.load(open(INDEX_MAP_EN_FILENAME, 'rb'))

Para facilitar a apresentação dos dados no resultado, o autor criou um dicionário onde o id da página representa o índice e o nome da página representa o valor

In [ ]:
names = {i: name for name, i in index_map_copy.items()}
names[FIFA_WORLD_CUP_PAGE_ID]

'FIFA_World_Cup'

Por fim iremos utilizar o Power Method para encontrarmos as páginas com melhor rank da nossa base de dados.

In [ ]:
def show_ex(v):
    print(',\n'.join(names[i] for i in np.abs(v.squeeze()).argsort()[-1:-10:-1]))
    
def power_method(A, max_iter=100):
    n = A.shape[1]
    A.data /= np.take(A.sum(axis=0).A1, A.indices)

    scores = np.ones(n, dtype=np.float32) * np.sqrt(A.sum()/(n*n)) # initial guess
    for i in range(max_iter):
        scores = A @ scores
        nrm = np.linalg.norm(scores)
        scores /= nrm

    return scores

In [ ]:
scores = power_method(sparse_matrix_csr)
show_ex(scores)

France,
Catholic_Church,
Africa,
Philosophy,
Ethnic_group,
Christian,
Buddhism,
Computer,
Plato


## 5.2 Utilizando base de dados PT-BR

Antes de aplicar o Power Method, precisamos refazer as etapas de download e manipulação, para obtermos os dados na estrutura esperada pela função

In [ ]:
PT_URL_BASE = 'http://downloads.dbpedia.org/3.8/pt/'
PT_REDIRECTS_FILENAME = 'redirects_pt.nt.bz2'
PT_PAGE_LINKS_FILENAME = 'page_links_pt.nt.bz2'

PT_DBPEDIA_RESOURCE_PREFIX_LEN = len("http://dbpedia.org/resource/")
PT_SLICE = slice(PT_DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)

pt_filenames = [PT_REDIRECTS_FILENAME, PT_PAGE_LINKS_FILENAME]

### Baixando os dados

In [ ]:
for filename in pt_filenames:
    if not os.path.exists(filename):
        print("Downloading '%s', please wait..." % filename)
        open(filename, 'wb').write(urlopen(PT_URL_BASE+filename).read())

### Organizando os dados e montando as estruturas esperadas

In [ ]:
pt_redirects = get_redirects(PT_REDIRECTS_FILENAME)

0it [00:00, ?it/s]

In [ ]:
# Computing the integer index map
pt_index_map = dict()  # links->IDs
pt_lines = get_lines(PT_PAGE_LINKS_FILENAME)
pt_source, pt_destination, pt_data = [], [], []
cont = 0
for l, split in tqdm.tqdm(enumerate(pt_lines)):
    add_item(pt_source, pt_redirects, pt_index_map, split[0].decode("utf-8"))
    add_item(pt_destination, pt_redirects, pt_index_map, split[2].decode("utf-8"))
    pt_data.append(1)

    # Limitando iterações devido a grande quantidade de dados
    if cont == 1_000_000:
      break
    cont += 1

len(pt_index_map)

0it [00:00, ?it/s]

234657

Dessa vez, o dicionário final contou com cerca de 234 mil índices.

Criando uma cópia dos dados para análise

In [ ]:
pt_index_map_copy = pt_index_map.copy()

Criando a matriz esparsa

In [ ]:
pt_n=len(pt_data)
pt_sparse_matrix = sparse.coo_matrix((pt_data, (pt_destination,pt_source)), shape=(pt_n,pt_n), dtype=np.float32)
pt_sparse_matrix_csr = pt_sparse_matrix.tocsr()

Métodos para salvar e carregar os dados obtidos (caso necessário)

In [ ]:
# SPARSE_MATRIX_PT_FILENAME = 'sparse_matrix_PT.pkl'
# INDEX_MAP_PT_FILENAME = 'index_map_PT.pkl'

# pickle.dump(pt_sparse_matrix_csr, open(SPARSE_MATRIX_PT_FILENAME, 'wb'))
# pickle.dump(pt_index_map_copy, open(INDEX_MAP_PT_FILENAME, 'wb'))

In [ ]:
# pt_sparse_matrix_csr = pickle.load(open(SPARSE_MATRIX_PT_FILENAME, 'rb'))
# pt_index_map_copy = pickle.load(open(INDEX_MAP_PT_FILENAME, 'rb'))

Criando dicionário chave valor para armazenar id e nome da página

In [ ]:
pt_names = {i: name for name, i in pt_index_map_copy.items()}

Por fim, utilizando o Power Method para encontrarmos as páginas com melhor rank.

In [ ]:
def show_ex(v):
    print(',\n'.join(pt_names[i].replace('ce/', '') for i in np.abs(v.squeeze()).argsort()[-1:-10:-1]))
    
def power_method(A, max_iter=100):
    n = A.shape[1]
    A.data /= np.take(A.sum(axis=0).A1, A.indices)

    scores = np.ones(n, dtype=np.float32) * np.sqrt(A.sum()/(n*n)) # initial guess
    for i in range(max_iter):
        scores = A @ scores
        nrm = np.linalg.norm(scores)
        scores /= nrm

    return scores

In [ ]:
pt_scores = power_method(pt_sparse_matrix_csr)
show_ex(pt_scores)

Orix\u00E1,
Roma_Antiga,
Computador,
Gr\u00E9cia_Antiga,
Anexo:Lista_de_reis_de_Portugal,
Eukaryota,
M\u00E9dio_Oriente,
Folha,
Pedro_I_do_Brasil
